In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("HScomcom/gpt2-game-of-thrones")

model = AutoModelForCausalLM.from_pretrained("HScomcom/gpt2-game-of-thrones")

: 

: 

In [2]:
text = "Once upon a time,"

encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

print(len(output))

2


In [7]:
input_ids = encoded_input.input_ids
outputs = model.generate(input_ids, do_sample=True, max_length=30, temperature=1.2)
res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(res)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["Once upon a time, I was a knight. Back in my day, you weren't a knight unless you killed a certain number of men. Then"]


In [2]:
from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from nltk.tokenize import sent_tokenize 
import numpy as np

In [4]:


class LanguageModelingDataset(Dataset):
    
    def __init__(self, ds_choice, split="train", truncation=-1):
        
        base_path = "../data/"
        self.ds_choice = ds_choice
        self.truncation = truncation  # int. If -1, then
        text = Path(base_path, ds_choice).read_text()
        if ds_choice == "large":
            # Remove the newline char in the middle of sentences
            # The "paragraph splitting" newlines appear to be \n\n -- remove the duplications there
            text = text.replace("\n\n", "$$^^$$").replace("\n", " ").replace("$$^^$$", "\n")
        sentences = sent_tokenize(text)

        # Train / test split
        train, val = train_test_split(sentences, test_size=0.2, shuffle=False)
        if split == "train":
            raw_data = train 
        else:
            raw_data = val 

        # Tokenize
        self.tokenizer = tokenizer()
        self.data = []  # List of 1-d pytorch tensor
        for sent in raw_data:
            tokenized = self.tokenizer(sent).view(-1)  # pytorch tensor
            if truncation >= 0:
                self.data.append(tokenized[:truncation])
            else:
                self.data.append(tokenized)

        # Count some items
        self.max_sentence_length = np.max([len(d) for d in self.data])

    def __len__(self):
        return len(self.data)

    def get_vocab_size(self):
        """
        We have to set this to the max vocab size (i.e., that decided by the BPE tokenizer), 
        but actually, only a small number of vocab is used, especially for the small text. 
        """
        return 50257

    def __getitem__(self, idx):
        """
        The output should be a tuple x and y, both as pytorch tensors.
        Please refer to the `run()` method in the mingpt/trainer.py script for 
        how the x and y are going to be used.
        """
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return (x, y)

    def get_block_size(self):
        """
        block_size is the size at which lines are truncated to ensure they are equal-length.
        """
        return self.max_sentence_length
    
# Instantiate the Training Dataset
#train_dataset = LanguageModelingDataset(ds_choice="small", split="train")  # use this for the short corpus
train_dataset = LanguageModelingDataset(ds_choice="Jon_1.txt", split="train", truncation=512) #use this for long

# Instantiate a Validation Dataset (this is only really needed for the fine-tune task, not the LM task)
#val_dataset = LanguageModelingDataset(ds_choice="small", split="validation")
val_dataset = LanguageModelingDataset(ds_choice="Jon_1.txt", split="validation", truncation=512)

NameError: name 'tokenizer' is not defined